In [1]:
import pandas as pd
import import_ipynb
import cdt_sql_server as cdtsql
from datetime import date
import time

importing Jupyter notebook from cdt_sql_server.ipynb


In [2]:
infoDataBases = {"ip":"",
                 "base":""}

In [3]:
def validaMovimentoCorte(infoDataBases, data):
    consulta1 = f"""
                SELECT * FROM ControleProcessos WHERE DataMovimento > '{data}' and FlagCorte = 1
                """
    consulta2 = f"""
                SELECT * FROM ControleVencimentos Where DataPrevistaCorte > '{data}'
                """
    controleprocessos = cdtsql.extrairDados(infoDataBases, consulta1, spark=False)
    controlevencimentos = cdtsql.extrairDados(infoDataBases, consulta2, spark=False)

    # Converte do formato o formato de data removendo o horario hh:mm:ss (Para prevenir mesmas datas com horarios diferentes de 00:00:00)
    controleprocessos['VencimentoPadraoFaturar'] = pd.to_datetime(controleprocessos['VencimentoPadraoFaturar'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
    controleprocessos['DataMovimento'] = controleprocessos['DataMovimento'].dt.strftime('%Y-%m-%d')
    controlevencimentos['DataPrevistaCorte'] = controlevencimentos['DataPrevistaCorte'].dt.strftime('%Y-%m-%d')
    controlevencimentos['DataVencimento'] = controlevencimentos['DataVencimento'].dt.strftime('%Y-%m-%d')
    controlevencimentos['DataPrevistaFaturamento'] = controlevencimentos['DataPrevistaFaturamento'].dt.strftime('%Y-%m-%d')
    controlevencimentos['DataPrevistaVencimento'] = controlevencimentos['DataPrevistaVencimento'].dt.strftime('%Y-%m-%d')
    
    result = {'Descricao': [],
              'Id_ControleProcesso': [],
              'Id_ControleVencimentos':[]}
    
    for i in range (len(controleprocessos)):
        #Verifica se a data de movimento esta inserida na ControleVencimento 
        df_aux = controlevencimentos[controleprocessos['DataMovimento'][i] == controlevencimentos['DataPrevistaCorte']]
        
        #Se não estiver
        if(len(df_aux) == 0):
            result['Descricao'].append('Data de Movimento Corte não foi encontrada na tabela ControleVencimentos')
            result['Id_ControleProcesso'].append(controleprocessos['Id_ControleProcesso'][i])
            result['Id_ControleVencimentos'].append(None)
            
        else:
            #Valida se os demais campos estão corretos
            if(((df_aux['DataVencimento'] == controleprocessos['VencimentoPadraoFaturar'][i]) == False).any()):
                result['Descricao'].append('Campo DataVencimento não foi validado na tabela ControleVencimentos')
                result['Id_ControleProcesso'].append(controleprocessos['Id_ControleProcesso'][i])
                result['Id_ControleVencimentos'].append(df_aux['Id_ControleVencimentos'][i])
                
            if(((df_aux['DataPrevistaFaturamento'] == controleprocessos['DataMovimento'][i]) == False).any()):
                result['Descricao'].append('Campo DataPrevistaFaturamento não foi validado na tabela ControleVencimentos')
                result['Id_ControleProcesso'].append(controleprocessos['Id_ControleProcesso'][i])
                result['Id_ControleVencimentos'].append(df_aux['Id_ControleVencimentos'][i])
                
            if(((df_aux['DataPrevistaVencimento'] == controleprocessos['VencimentoPadraoFaturar'][i]) == False).any()):
                result['Descricao'].append('Campo VencimentoPadraoFaturar não foi validado na tabela ControleVencimentos')
                result['Id_ControleProcesso'].append(controleprocessos['Id_ControleProcesso'][i])
                result['Id_ControleVencimentos'].append(df_aux['Id_ControleVencimentos'][i])
    
    return pd.DataFrame(result)
        

# Tempo de Execução da Função

In [24]:
ini = time.time()
resultado = validaMovimentoCorte(infoDataBases, date.today())
fim = time.time()
print ("Tempo em segundos : ", fim-ini)

Tempo em segundos :  4.659976482391357


# ----------------------------------------------------------------------------

# Casos de Testes

## Função teste sem leitura da base

In [5]:
def teste(infoDataBases, data):
    # Converte do formato o formato de data removendo o horario hh:mm:ss (Para prevenir mesmas datas com horarios diferentes de 00:00:00)
    
    result = {'Descricao': [],
              'Id_ControleProcesso': [],
              'Id_ControleVencimentos': []}
    
    for i in range (len(controleprocessos)):
        #Verifica se a data de movimento esta inserida na ControleVencimento 
        df_aux = controlevencimentos[controleprocessos['DataMovimento'][i] == controlevencimentos['DataPrevistaCorte']]
        
        #Se não estiver
        if(len(df_aux) == 0):
            result['Descricao'].append('Data de Movimento Corte não foi encontrada na tabela ControleVencimentos')
            result['Id_ControleProcesso'].append(controleprocessos['Id_ControleProcesso'][i])
            result['Id_ControleVencimentos'].append('None')
            
        else:
            #Valida se os demais campos estão corretos
            if(((df_aux['DataVencimento'] == controleprocessos['VencimentoPadraoFaturar'][i]) == False).any()):
                result['Descricao'].append('Campo DataVencimento não foi validado na tabela ControleVencimentos')
                result['Id_ControleProcesso'].append(controleprocessos['Id_ControleProcesso'][i])
                result['Id_ControleVencimentos'].append(int(df_aux['Id_ControleVencimentos'][i]))
                
            if(((df_aux['DataPrevistaFaturamento'] == controleprocessos['DataMovimento'][i]) == False).any()):
                result['Descricao'].append('Campo DataPrevistaFaturamento não foi validado na tabela ControleVencimentos')
                result['Id_ControleProcesso'].append(controleprocessos['Id_ControleProcesso'][i])
                result['Id_ControleVencimentos'].append(int(df_aux['Id_ControleVencimentos'][i]))
                
            if(((df_aux['DataPrevistaVencimento'] == controleprocessos['VencimentoPadraoFaturar'][i]) == False).any()):
                result['Descricao'].append('Campo DataPrevistaVencimento não foi validado na tabela ControleVencimentos')
                result['Id_ControleProcesso'].append(controleprocessos['Id_ControleProcesso'][i])
                result['Id_ControleVencimentos'].append(int(df_aux['Id_ControleVencimentos'][i]))
    
    return pd.DataFrame(result)

In [6]:
#Leitura da base
data = date.today()
consulta1 = f"""
            SELECT * FROM ControleProcessos WHERE DataMovimento > '{data}' and FlagCorte = 1
            """
consulta2 = f"""
            SELECT * FROM ControleVencimentos Where DataPrevistaCorte > '{data}'
            """
controleprocessos = cdtsql.extrairDados(infoDataBases, consulta1, spark=False)
controlevencimentos = cdtsql.extrairDados(infoDataBases, consulta2, spark=False)

controleprocessos['VencimentoPadraoFaturar'] = pd.to_datetime(controleprocessos['VencimentoPadraoFaturar'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
controleprocessos['DataMovimento'] = controleprocessos['DataMovimento'].dt.strftime('%Y-%m-%d')
controlevencimentos['DataPrevistaCorte'] = controlevencimentos['DataPrevistaCorte'].dt.strftime('%Y-%m-%d')
controlevencimentos['DataVencimento'] = controlevencimentos['DataVencimento'].dt.strftime('%Y-%m-%d')
controlevencimentos['DataPrevistaFaturamento'] = controlevencimentos['DataPrevistaFaturamento'].dt.strftime('%Y-%m-%d')
controlevencimentos['DataPrevistaVencimento'] = controlevencimentos['DataPrevistaVencimento'].dt.strftime('%Y-%m-%d')

# 1- Dado faltante

In [7]:
# Removendo uma instância de controleVencimentos
#Removendo instancia 2
controlevencimentos.drop(2, inplace = True)

In [8]:
resultado = teste(infoDataBases, data)

In [9]:
resultado

,Descricao,Id_ControleProcesso,Id_ControleVencimentos
0,Data de Movimento Corte não foi encontrada na ...,7464,None


# 2- DataVencimento não validado

In [10]:
controlevencimentos.loc[50,'DataVencimento'] = None

In [11]:
resultado = teste(infoDataBases, data)

In [12]:
resultado

,Descricao,Id_ControleProcesso,Id_ControleVencimentos
0,Data de Movimento Corte não foi encontrada na ...,7464,None
1,Campo DataVencimento não foi validado na tabel...,7709,1921


# 3- DataPrevistaFaturamento não validado

In [13]:
controlevencimentos.loc[15,'DataPrevistaFaturamento'] = None

In [14]:
resultado = teste(infoDataBases, data)

In [15]:
resultado

,Descricao,Id_ControleProcesso,Id_ControleVencimentos
0,Data de Movimento Corte não foi encontrada na ...,7464,None
1,Campo DataPrevistaFaturamento não foi validado...,7532,1886
2,Campo DataVencimento não foi validado na tabel...,7709,1921


# 3- DataPrevistaVencimento não validado

In [16]:
controlevencimentos.loc[10,'DataPrevistaVencimento'] = '1999-12-12'

In [17]:
resultado = teste(infoDataBases, data)

In [18]:
resultado

,Descricao,Id_ControleProcesso,Id_ControleVencimentos
0,Data de Movimento Corte não foi encontrada na ...,7464,None
1,Campo DataPrevistaVencimento não foi validado ...,7506,1881
2,Campo DataPrevistaFaturamento não foi validado...,7532,1886
3,Campo DataVencimento não foi validado na tabel...,7709,1921


# Mais alguns testes

In [19]:
controlevencimentos.loc[57,'DataPrevistaVencimento'] = '191-12-12'

In [20]:
controlevencimentos.loc[10,'DataPrevistaFaturamento'] = '1999-12-12'

In [21]:
controleprocessos.loc[10,'DataMovimento'] = '1111-10-10'

In [22]:
resultado = teste(infoDataBases, data)

In [23]:
resultado

,Descricao,Id_ControleProcesso,Id_ControleVencimentos
0,Data de Movimento Corte não foi encontrada na ...,7464,None
1,Data de Movimento Corte não foi encontrada na ...,7506,None
2,Campo DataPrevistaFaturamento não foi validado...,7532,1886
3,Campo DataVencimento não foi validado na tabel...,7709,1921
4,Campo DataPrevistaVencimento não foi validado ...,7744,1928
